In [1]:
# Data annotations described here:
# https://www.mpi-inf.mpg.de/departments/computer-vision-and-machine-learning/software-and-datasets/mpii-human-pose-dataset/download
import scipy.io

mat_file_path = 'content/mpii_human_pose_v1_u12_2/mpii_human_pose_v1_u12_1.mat'
release_data = scipy.io.loadmat(mat_file_path)['RELEASE']
print(f"Fields inside 'RELEASE' structured array: {release_data.dtype.names}")
annolist = release_data['annolist'][0, 0]
print(f"Shape of 'annolist': {annolist.shape}")

first_image_annots = annolist[0, 0]  # Or maybe annolist[0] depending on shape
print("\nInspecti----", first_image_annots.dtype.names)
print(first_image_annots['image'][0][0][0][0])

act = release_data['act'][0, 0]
print(f"act ::: {act}")
filenames_map = {}

for datum in range(annolist.shape[1]):  # Iterate through the second dimension of annolist
    image_annot = annolist[0, datum]  # Access the element using index i in the second dimension
    filename = image_annot['image'][0][0][0][0]  # Extract the filename

    filenames_map[datum] = filename

    if datum < 5:
        print(f"Filename for index {datum}: {filename}")

Fields inside 'RELEASE' structured array: ('annolist', 'img_train', 'version', 'single_person', 'act', 'video_list')
Shape of 'annolist': (1, 24987)

Inspecti---- ('image', 'annorect', 'frame_sec', 'vididx')
037454012.jpg
act ::: [[(array([], dtype='<U1'), array([], dtype='<U1'), array([[-1]], dtype=int16))]
 [(array([], dtype='<U1'), array([], dtype='<U1'), array([[-1]], dtype=int16))]
 [(array([], dtype='<U1'), array([], dtype='<U1'), array([[-1]], dtype=int16))]
 ...
 [(array(['transportation'], dtype='<U14'), array(['pushing car'], dtype='<U11'), array([[972]], dtype=uint16))]
 [(array([], dtype='<U1'), array([], dtype='<U1'), array([[-1]], dtype=int16))]
 [(array([], dtype='<U1'), array([], dtype='<U1'), array([[-1]], dtype=int16))]]
Filename for index 0: 037454012.jpg
Filename for index 1: 095071431.jpg
Filename for index 2: 073199394.jpg
Filename for index 3: 059865848.jpg
Filename for index 4: 015601864.jpg


In [2]:
import os

act = release_data['act'][0, 0]
dataset = []
number_skipped = 0
activities_set = set()

for datum, item in enumerate(act):
    try:
        cat_name = item[0][0][0] if len(item[0]) > 0 and len(item[0][0]) > 0 else None
        act_name = item[0][1][0] if len(item[0]) > 1 and len(item[0][1]) > 0 else None
        activities = str(act_name).split(", ")
        #act_id = item[0][2][0][0] if len(item[0]) > 2 and len(item[0][2]) > 0 else None
        if not act_name:
            number_skipped += 1
            continue
    except IndexError:
        print("IndexError encountered. Skipping this item.")
        continue

    filename = "content/images/" + str(filenames_map[datum])
    if os.path.exists(filename):
        dataset.append((filename, activities))
        activities_set.update(activities)
    else:
        print(f"File {filename} does not exist. Skipping this item.")
        number_skipped += 1

print(f"number skipped: {number_skipped}")

# Žiūrim ką turim. 🥸
for i, datum in enumerate(dataset):
    filename, act_name = datum
    if i > 100:
        break
    print(f"I: {i}. Filename {filename} act Name: {act_name}")

File content/images/040348287.jpg does not exist. Skipping this item.
File content/images/013401523.jpg does not exist. Skipping this item.
File content/images/002878268.jpg does not exist. Skipping this item.
number skipped: 6957
I: 0. Filename content/images/015601864.jpg act Name: ['curling']
I: 1. Filename content/images/015599452.jpg act Name: ['curling']
I: 2. Filename content/images/005808361.jpg act Name: ['curling']
I: 3. Filename content/images/086617615.jpg act Name: ['curling']
I: 4. Filename content/images/060111501.jpg act Name: ['curling']
I: 5. Filename content/images/070807258.jpg act Name: ['curling']
I: 6. Filename content/images/002058449.jpg act Name: ['curling']
I: 7. Filename content/images/021233911.jpg act Name: ['sitting quietly']
I: 8. Filename content/images/018182497.jpg act Name: ['sitting quietly']
I: 9. Filename content/images/018340451.jpg act Name: ['sitting', 'talking in person', 'on the phone', 'computer', 'or text messaging', 'light effort']
I: 10. 

In [3]:
activities_set

{' carving wood',
 '(e.g.',
 '2.5 to 3.1 mph',
 '295)',
 '4-square',
 '75 lbs or more',
 'Alaska Native Games',
 'Anishinaabe Jingle Dancing',
 'BMX',
 'Beguine',
 'Bellair',
 'Bongo',
 "Brukin's",
 'Caribbean Quadrills',
 'Caribbean dance (Abakua',
 'Dinki Mini',
 'Elliptical trainer',
 'Eskimo Olympics',
 'Fitball exercise',
 'G',
 'Gere',
 'Greek',
 'Irish step dancing',
 'Middle Eastern',
 'Nadisodhana',
 'Navy Seal',
 'Power',
 'TV conditioning programs',
 'accordion',
 'active workstation',
 'adults playing (e.g.',
 'aerobic',
 'airline flight attendant',
 'all-terrain vehicle',
 'andor with talking involved (e.g.',
 'applying fertilizer or seeding a lawn',
 'archery',
 'arts and crafts',
 'assisting with birthing,',
 'automobile repair',
 'backpacking',
 'backstage employee',
 'backstroke',
 'badminton',
 'bakery',
 'ballet',
 'ballroom',
 'bamba y plena',
 'base jumping',
 'basketball',
 'batting',
 'beach',
 'benbe)',
 'biathlon',
 'bicycling',
 'billiards',
 'bird watching',


In [4]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

from sklearn.preprocessing import LabelEncoder

# Create a label encoder for activities
label_encoder = LabelEncoder()
all_activities = list(activities_set)
label_encoder.fit(all_activities)


# Update the MPIIDataset class
class MPIIDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, act_name = self.data[idx]
        image = Image.open(image_path).convert('RGB')  # Ensure RGB format

        if self.transform:
            image = self.transform(image)

        # Convert activity name(s) to numerical label(s)
        label = label_encoder.transform(act_name)  # Convert to numerical indices
        label = label[0] if len(label) > 0 else 0  # Handle single-label case

        return image, label


# Split the dataset into train, validation, and test sets
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

train_size = int(len(dataset) * train_ratio)
val_size = int(len(dataset) * val_ratio)
test_size = len(dataset) - train_size - val_size

train_dataset = []
val_dataset = []
test_dataset = []

train_size = int(len(dataset) * train_ratio)
val_size = int(len(dataset) * val_ratio)

for i in range(len(dataset)):
    if i < train_size:
        train_dataset.append(dataset[i])
    elif i < train_size + val_size:
        val_dataset.append(dataset[i])
    else:
        test_dataset.append(dataset[i])

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

transform = transforms.Compose([
    transforms.Resize(256),  # Resize the images
    transforms.CenterCrop(224),  # Crop the images
    transforms.ToTensor(),  # Convert to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the images
])

train_dataset = MPIIDataset(train_dataset, transform=transform)
val_dataset = MPIIDataset(val_dataset, transform=transform)
test_dataset = MPIIDataset(test_dataset, transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


Train dataset size: 14424
Validation dataset size: 1803
Test dataset size: 1803


In [5]:
import torch

# Because not 42 🤮
random_state = 137  # 🥰

NUM_EPOCHS = 55

DEVICE = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Using device: cuda


In [6]:
import torch.nn as nn


class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(128 * 56 * 56, 256),  # Adjust input size based on image dimensions after conv layers.
            nn.ReLU(inplace=True),
            nn.Linear(256, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [14]:
def train_model(train_loader, val_loader, test_loader, model_class=SimpleCNN, epochs=NUM_EPOCHS):
    import torch.optim as optim
    from tqdm import tqdm
    import matplotlib.pyplot as plt

    # Assuming you have num_classes
    num_classes = len(activities_set)

    # Initialize the model, loss function, and optimizer
    model = model_class(num_classes=len(activities_set))
    model.to(DEVICE)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    best_val_acc = 0

    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        train_correct = 0
        train_total = 0

        for images, labels in tqdm(train_loader):  # Use tqdm for progress bar
            images, labels = images.to(DEVICE), labels.to(DEVICE)  # Move data to the device
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            predicted = torch.argmax(outputs, dim=1)  # Get the predicted class indices
            train_correct += (predicted == labels).sum().item()  # Compare with labels
            train_total += labels.size(0)  # Track the total number of samples

            # Print progress (optional)
            if epoch % 5 == 0:
                print(f'Epoch [{epoch + 1}/{epochs}], Batch [{epoch + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

              # Calculate and append training accuracy for this epoch
        train_accuracy = 100 * train_correct / train_total
        train_accuracies.append(train_accuracy)

        # Validation loop (modified)
        model.eval()  # Set the model to evaluation mode
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
          for data, target in val_loader:
              # Move data to the selected device
              data, target = data.to(DEVICE), target.to(DEVICE)
              outputs = model(data)
              loss = criterion(outputs, target)
              val_loss += loss.item()

              predicted = (outputs > 0.5).float()
              total += target.size(0)
              correct += (predicted == target).sum().item()

        val_loss /= len(val_loader)  # Average validation loss
        val_accuracy = 100 * correct / total

        train_losses.append(loss.item()) #for plotting
        val_losses.append(val_loss) #for plotting
        #train_accuracies.append((100 * correct) / total) #for plotting
        val_accuracies.append(val_accuracy) #for plotting

        if val_accuracy > best_val_acc:
            best_val_acc = val_accuracy
            best_weights =  model.state_dict()
            print(f"Saving best model...💾")
            torch.save(best_weights, "best.pt")

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {running_loss / len(train_loader)}")
        print(f'Epoch [{epoch+1}/{epochs}], Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

    # Plot the loss and accuracy curves
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Loss Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label='Training Accuracy')
    plt.plot(val_accuracies, label='Validation Accuracy')
    plt.title('Accuracy Curves')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()

    plt.show()
    return best_val_acc


import torch
torch.cuda.empty_cache()
train_model(train_loader, val_loader, test_loader, model_class=SimpleCNN, epochs=5)

OutOfMemoryError: CUDA out of memory. Tried to allocate 392.00 MiB. GPU 0 has a total capacity of 7.78 GiB of which 294.69 MiB is free. Including non-PyTorch memory, this process has 6.62 GiB memory in use. Of the allocated memory 6.42 GiB is allocated by PyTorch, and 33.23 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)